# Exploring Initial Datasets for Project
This notebook is for will importing key modules and beginning to explore data for final MS Project.

In [6]:
import numpy, matplotlib, pandas, geopandas, pandas, fiona

In [14]:
# load census designated places shapefile
cdps_raw = geopandas.read_file("C:/NewMapsPlus/Map698/us-communities-fire/data/cb_2020_us_place_500k.zip")

In [15]:
# inspect
type(cdps_raw)

geopandas.geodataframe.GeoDataFrame

In [16]:
cdps_raw.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 32188 entries, 0 to 32187
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   STATEFP     32188 non-null  object  
 1   PLACEFP     32188 non-null  object  
 2   PLACENS     32188 non-null  object  
 3   AFFGEOID    32188 non-null  object  
 4   GEOID       32188 non-null  object  
 5   NAME        32188 non-null  object  
 6   NAMELSAD    32188 non-null  object  
 7   STUSPS      32188 non-null  object  
 8   STATE_NAME  32188 non-null  object  
 9   LSAD        32188 non-null  object  
 10  ALAND       32188 non-null  int64   
 11  AWATER      32188 non-null  int64   
 12  geometry    32188 non-null  geometry
dtypes: geometry(1), int64(2), object(10)
memory usage: 3.2+ MB


In [17]:
cdps_raw.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - NAD83
- bounds: (167.65, 14.92, -47.74, 86.46)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich